[View in Colaboratory](https://colab.research.google.com/github/juglar-diaz/STTD/blob/master/Represent.ipynb)

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a1b6000 @  0x7fb97dcc71c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 17.0MB/s 
    100% |████████████████████████████████| 2.0MB 2.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
from collections import Counter
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import itertools

import torch.nn as nn
import torch.optim as optim
import random
torch.manual_seed(1)


import pickle

import os
sep = os.sep
import os.path

data = ""
import pandas as pd
import bisect
import time

In [0]:
if torch.cuda.is_available():
    inGPU = True
    _type = torch.cuda.FloatTensor
    _typelong = torch.cuda.LongTensor
    device = torch.device("cuda")
else:
    inGPU = False
    _type = torch.FloatTensor
    _typelong = torch.LongTensor
    device = torch.device("cuda")
    

#Data

In [0]:
exchangedrive = drive.CreateFile({'id':'1cMCzlvTMlUPgaYaUIdlMVtTpp8r10GnX'})
exchangedrive.GetContentFile('robosclean.p')

In [0]:
exchangedrive = drive.CreateFile({'id':'1A5Wa6LiaGs8XeW2S2qjGbcoSSlsMGW97'})
exchangedrive.GetContentFile('tweetsLAtrain.csv')
exchangedrive = drive.CreateFile({'id':'1CrUCS7oWzdvYoFwtWgikiGkDAu6w3dOF'})
exchangedrive.GetContentFile('tweetsLAtest.csv')

In [0]:
exchangedrive = drive.CreateFile({'id':'1NuSVM7-h0CCRtzi0woM4h5tVd6T7JO2C'})
exchangedrive.GetContentFile('tweetsNYtrain.csv')
exchangedrive = drive.CreateFile({'id':'1UYZY0sh1-Q8MIofMAHDGaNKv8cfDA0ui'})
exchangedrive.GetContentFile('tweetsNYtest.csv')

In [0]:
exchangedrive = drive.CreateFile({'id':'1Jv19eJTZwsZWEA_rUudvKwcn1TS-DFBa'})
exchangedrive.GetContentFile('tweets2016_2half.csv')
exchangedrive = drive.CreateFile({'id':'1E8WlhOXb3tfQbLUpizx7QXzeydv57eXN'})
exchangedrive.GetContentFile('toy_2017_Jan.csv')

In [0]:
def buildIndexData(list_elements, start_index = 0):

    idx2data = {index + start_index: discretization for index, discretization in enumerate(set(list_elements))}

    data2idx = {discretization: index for index, discretization in idx2data.items()}

    indexes = [data2idx[element] for element in list_elements]

    return indexes, data2idx, idx2data


class Discretize:
    def fit_transform(self):
        pass

    def transform(self):
        pass

    def updateIndexes(self, indexes, star_index):
        map_indexes = {value:counter+star_index for counter, value in enumerate(set(indexes))}


        new_indexes = [map_indexes[index] for index in indexes]

        idx2data = {map_indexes[index]:self.idx_data[index] for index in set(indexes)}
        data2idx = {val:key for (key, val) in idx2data.items()}

        self.idx_data = idx2data
        self.data_idx = data2idx

        return new_indexes





In [0]:
class Round(Discretize):
    def __init__(self, div= 10):
        self.div= div

    def fit_transform(self, latitudes, longitudes, start_index = 0, vocab_size = -1, vocab_min_count=0):
        lat = (latitudes * self.div).astype(int)
        lon = (longitudes * self.div).astype(int)

        discretizations = list(zip(lat, lon))

        counter = Counter(discretizations)
        if (vocab_size > 0):
            pairs = counter.most_common(vocab_size)
        else:
            pairs = list(counter.items())

        self.vocab = [keyword for keyword, count in pairs if count >= vocab_min_count]

        self.indexes, self.data_idx, self.idx_data = buildIndexData(self.vocab, start_index)



        return [self.data_idx.get(data, None) for data in discretizations], self.data_idx, self.idx_data



    def transform(self, latitudes, longitudes):
        lat = (latitudes * self.div).astype(int)
        lon = (longitudes * self.div).astype(int)

        discretizations = list(zip(lat, lon))
        #indexes = [self.data_idx.get(data, None) for data in discretizations]
        #return indexes
        return discretizations

In [0]:
class HourofDay(Discretize):
    def fit_transform(self, created_at, start_index = 0):
        indi = pd.DatetimeIndex(created_at)
        discretizations = list(indi.hour)
        self.indexes, self.data_idx, self.idx_data = buildIndexData(discretizations, start_index)
        return self.indexes, self.data_idx, self.idx_data


    def transform(self, created_at):
        indi = pd.DatetimeIndex(created_at)
        discretizations = list(indi.hour)
        #indexes = [self.data_idx.get(data,None) for data in discretizations]
        #return indexes
        return discretizations

class DayofWeak(Discretize):
    def fit_transform(self, created_at, start_index = 0):
        indi = pd.DatetimeIndex(created_at)

        discretizations = list(indi.weekday)
        self.indexes, self.data_idx, self.idx_data = buildIndexData(discretizations, start_index)
        return self.indexes, self.data_idx, self.idx_data

    def transform(self, created_at):
        indi = pd.DatetimeIndex(created_at)
        discretizations = list(indi.weekday)
        indexes = [self.data_idx.get(data,None) for data in discretizations]
        return indexes


class HourofDay_DayofWeak(Discretize):
    def fit_transform(self, created_at, start_index = 0):
        indi = pd.DatetimeIndex(created_at)

        discretizations = list(zip(indi.weekday, indi.hour))
        self.indexes, self.data_idx, self.idx_data = buildIndexData(discretizations, start_index)
        return self.indexes, self.data_idx, self.idx_data

    def transform(self, created_at):
        indi = pd.DatetimeIndex(created_at)

        discretizations = list(zip(indi.weekday, indi.hour))
        indexes = [self.data_idx.get(data, None) for data in discretizations]
        return indexes

In [0]:

class Indexer():
    def __init__(self):
        pass

    def fit_transform(self,
            filename,
            time_discretizer = HourofDay,
            coor_discretizer = Round,
            #represent_text = RepresentText,
            dates_vocab_size = 0, dates_vocab_mincount = 0,
            places_vocab_size = 0, places_vocab_mincount = 0,
            words_vocab_size = 0, words_vocab_mincount = 0): #file_csv has columns created_at, latitude, longitude, text

        #self.datapath = "Data" + sep
        if (filename.split('.')[1] == 'csv'):
            df = pd.read_csv(filename)
        elif (filename.split('.')[1] == 'p'):
            df = pd.read_pickle(filename)

        self.time_discretizer = time_discretizer()
        self.coor_discretizer = coor_discretizer(100)
        #self.represent_text = represent_text()



        #date_out, self.date_idx, self.idx_date = self.time_discretizer.fit_transform(df['created_at'], start_index = 0)
        dates = self.time_discretizer.transform(df['created_at'])

        #self.coor_out, self.coor_idx, self.idx_coor = self.coor_discretizer.fit_transform(df['latitude'], df['longitude'],start_index=max(self.date_out) + 1)

        places = self.coor_discretizer.transform(df['latitude'], df['longitude'])

        '''
        datapath = "SavedModels" + sep
        modelpath = datapath+"represent_text_"+ filename.split('.')[0]  +"words"+str(words_min_count)+".model"
        if os.path.isfile(modelpath):
            self.represent_text = pickle.load(open(datapath+"represent_text_"+ filename.split('.')[0]+ ".model", "rb"))

            self.texts_out = pickle.load(open(datapath+"texts_out"+ filename.split('.')[0]  +".p", "rb"))
            self.keyword_idx = pickle.load(open(datapath+"keyword_idx_"+ filename.split('.')[0]  +".p", "rb"))
            self.idx_keyword = pickle.load(open(datapath+"idx_keyword_"+ filename.split('.')[0]  +".p", "rb"))


        else:
            self.texts_out, self.keyword_idx, self.idx_keyword = self.represent_text.fit_transform(df['texts'], start_index= max(self.coor_out) + 1, vocab_min_count= words_min_count)

            pickle.dump( self.represent_text, open(modelpath, "wb" ), pickle.HIGHEST_PROTOCOL)


            pickle.dump( self.texts_out, open(datapath+"texts_out"+ filename.split('.')[0]+ ".p", "wb" ))
            pickle.dump( self.keyword_idx, open(datapath+"keyword_idx_"+ filename.split('.')[0]+ ".p", "wb" ) )
            pickle.dump( self.idx_keyword, open(datapath+"idx_keyword_"+ filename.split('.')[0]+ ".p", "wb" ) )

        '''
        texts = df['texts'].astype(str)
        words = [word for list_words in texts for word in list_words.split()]
        #print(len(texts))


        counter_dates = Counter(dates)
        if (dates_vocab_size > 0):
            pairs = counter_dates.most_common(dates_vocab_size)
        else:
            pairs = list(counter_dates.items())
        self.vocab_dates = set([date for date, count in pairs if count >= dates_vocab_mincount])


        counter_places = Counter(places)
        if (places_vocab_size > 0):
            pairs = counter_places.most_common(places_vocab_size)
        else:
            pairs = list(counter_places.items())
        self.vocab_places = set([place for place, count in pairs if count >= places_vocab_mincount])


        counter_words = Counter(words)
        if (words_vocab_size > 0):
            pairs = counter_words.most_common(words_vocab_size)
        else:
            pairs = list(counter_words.items())

        self.vocab_words = set([keyword for keyword, count in pairs if count >= words_vocab_mincount])




        filtered_dates = set([i for i in range(len(dates)) if dates[i] in self.vocab_dates ])
        filtered_places = set([i for i in range(len(places)) if places[i] in self.vocab_places])
        filtered_words = set([i for i in range(len(texts)) if any([word in self.vocab_words for word in texts[i].split()]) ])

        #print(len(filtered_dates))
        #print(len(filtered_places))
        #print(len(filtered_words))



        filtered = list(filtered_dates.intersection(filtered_places).intersection(filtered_words))
        #print(len(filtered))

        dates = [dates[i] for i in filtered]
        places = [places[i] for i in filtered]
        texts = [texts[i] for i in filtered]


        idxsdates, self.date2idx, self.idx2date = buildIndexData(dates, start_index=0)
        idxsplaces, self.place2idx, self.idx2place = buildIndexData(places, start_index=max(idxsdates) + 1)




        idxs, self.word2idx, self.idx2word = buildIndexData(self.vocab_words, start_index = max(idxsplaces) + 1)


        idxstexts = []

        for text in texts:
            indexed_text = [self.word2idx[word] for word in text.split() if word in self.vocab_words]
            idxstexts.append(indexed_text)


        self.idx2item = {}
        self.idx2item.update(self.idx2word)
        self.idx2item.update(self.idx2place)
        self.idx2item.update(self.idx2date)

        self.item2idx = {}
        self.item2idx.update(self.word2idx)
        self.item2idx.update(self.place2idx)
        self.item2idx.update(self.date2idx)

        return list(zip(idxsdates,
                             idxsplaces,
                             idxstexts))


    def transform(self, filename):
        if (filename.split('.')[1] == 'csv'):
            df = pd.read_csv( filename)
        elif (filename.split('.')[1] == 'p'):
            df = pd.read_pickle( filename)

        dates = self.time_discretizer.transform(df['created_at'])
        idxsdates = [self.date2idx.get(date, None) for date in dates]

        places = self.coor_discretizer.transform(df['latitude'], df['longitude'])
        idxsplaces = [self.place2idx.get(place, None) for place in places]

        idxstexts = []
        for text in df['texts'].astype(str):
            indexed_text = [self.word2idx[word] for word in text.split() if word in self.vocab_words]
            idxstexts.append(indexed_text)


        full_list = list(zip(idxsdates,
                             idxsplaces,
                             idxstexts))
        #print(len(full_list))
        clean_list = [(x[0], x[1], x[2]) for x in full_list if ((x[0] != None) and (x[1] != None) and (x[2] != [])) ]
        return clean_list


    def Item2index(self, item):
        return self.item2idx.get(item, -1)

    def Index2item(self, index):
        return self.idx2item.get(index, None)

    def indexes(self):
        return (self.coor_out[0], self.coor_out[-1], self.texts_out[-1])

In [14]:
%%time
indexerST = Indexer()
trainST = indexerST.fit_transform(data+'tweets2016_2half.csv', dates_vocab_mincount=0, words_vocab_mincount=100, places_vocab_mincount=10)
testST = indexerST.transform(data+'toy_2017_Jan.csv')

CPU times: user 11 s, sys: 248 ms, total: 11.3 s
Wall time: 11.3 s


In [15]:
%%time
indexerLA = Indexer()
trainLA = indexerLA.fit_transform(data+'tweetsLAtrain.csv', dates_vocab_mincount=0, words_vocab_mincount=100, places_vocab_mincount=10)
testLA = indexerLA.transform(data+'tweetsLAtest.csv')

CPU times: user 40.4 s, sys: 797 ms, total: 41.2 s
Wall time: 41.2 s


In [16]:
%%time
indexerNY = Indexer()
trainNY = indexerNY.fit_transform(data+'tweetsNYtrain.csv', dates_vocab_mincount=0, words_vocab_mincount=100, places_vocab_mincount=10)
testNY = indexerNY.transform(data+'tweetsNYtest.csv')

CPU times: user 11.4 s, sys: 101 ms, total: 11.5 s
Wall time: 11.5 s


In [0]:
class QuantitativeEvaluator:
    def __init__(self, fake_num=10):
        self.ranks = []

        self.fake_num = fake_num


    def get_ranks(self, test, predictor, predict_type = 'w'):
        self.predict_type = predict_type
        noiseList = np.random.choice(len(test), self.fake_num*len(test)).tolist()
        count = 5
        for example in test:



            scores = []
            score = predictor.predict(example[0], example[1], example[2])
            scores.append(score)


            for i in range(self.fake_num):
                noise = test[noiseList.pop()]
                if self.predict_type == 't':
                    noise_score = predictor.predict(noise[0], example[1], example[2])
                elif self.predict_type=='l':
                    noise_score = predictor.predict(example[0], noise[1], example[2])
                elif self.predict_type=='w':
                    noise_score = predictor.predict(example[0], example[1], noise[2])
                scores.append(noise_score)
            scores.sort()


            # handle ties
            rank = len(scores)+1-(bisect.bisect_left(scores,score)+bisect.bisect_right(scores,score)+1)/2.0
            self.ranks.append(rank)


    def get_ranks_old(self, np_xytest, predictor, predict_type = 'w'):
        self.predict_type = predict_type
        noiseList = np.random.choice(np_xytest.shape[0], self.fake_num*np_xytest.shape[0]).tolist()

        for i in range(np_xytest.shape[0]):
            example = list(np_xytest[i])
            scores = []
            score = predictor.predict(example[0], example[1], example[2])
            scores.append(score)

            for i in range(self.fake_num):
                noise = np_xytest[noiseList.pop()]
                if self.predict_type == 'l':
                    noise_score = predictor.predict(noise[0], example[1], example[2])
                elif self.predict_type=='t':
                    noise_score = predictor.predict(example[0], noise[1], example[2])
                elif self.predict_type=='w':
                    noise_score = predictor.predict(example[0], example[1], noise[2])
                scores.append(noise_score)
            scores.sort()
            # handle ties
            rank = len(scores)+1-(bisect.bisect_left(scores,score)+bisect.bisect_right(scores,score)+1)/2.0
            self.ranks.append(rank)

    def compute_mrr(self):
        r = self.ranks
        reciprocal_ranks = [1/rank for rank in r]
        mrr = sum(reciprocal_ranks)/len(reciprocal_ranks)
        mr = sum(r)/len(r)
        return round(mrr,4), round(mr,4)


class QualitativeEvaluator:
    def __init__(self, predictor, output_dir):
        self.predictor = predictor
        self.output_dir = output_dir

    def plot_locations_on_google_map(self, locations, output_path):
        request ='https://maps.googleapis.com/maps/api/staticmap?zoom=10&size=600x600&maptype=roadmap&'
        for lat, lng in locations:
            request += 'markers=color:red%7C' + '%f,%f&' % (lat, lng)
        proxy = urllib.ProxyHandler({})
        opener = urllib.build_opener(proxy)
        response = opener.open(request).read()
        with open(output_path, 'wb') as f:
            f.write(response)
            f.close()
        time.sleep(3)

    def scribe(self, directory, ls, ts, ws, show_ls):
        if not os.path.isdir(directory):
            os.makedirs(directory)
        for nbs, file_name in [(ts, 'times.txt'), (ws, 'words.txt')]:
            output_file = open(directory+file_name,'w')
            for nb in nbs:
                output_file.write(str(nb)+'\n')
        if show_ls:
            self.plot_locations_on_google_map(ls[:10], directory+'locations.png')
        else:
            self.plot_locations_on_google_map(ls[:1], directory+'queried_location.png')

    def getNbs1(self, query):
        if type(query)==str and query.lower() not in self.predictor.nt2vecs['w']:
            print (query, 'not in voca')
            return
        directory = self.output_dir+str(query)+'/'
        ls, ts, ws = [self.predictor.get_nbs1(query, nt) for nt in ['l', 't', 'w']]
        self.scribe(directory, ls, ts, ws, type(query)!=list)

    def getNbs2(self, query1, query2, func=lambda a, b:a+b):
        if type(query1)==str and query1.lower() not in self.predictor.nt2vecs['w']:
            print (query1, 'not in voca')
            return
        if type(query2)==str and query2.lower() not in self.predictor.nt2vecs['w']:
            print (query2, 'not in voca')
            return
        directory = self.output_dir+str(query1)+'-'+str(query2)+'/'
        ls, ts, ws = [self.predictor.get_nbs2(query1, query2, func, nt) for nt in ['l', 't', 'w']]
        self.scribe(directory, ls, ts, ws, type(query1)!=list)

In [28]:
t = torch.zeros([150,2,100])
print(t.shape)
print(torch.sum(t, dim=1).shape)

torch.Size([150, 2, 100])
torch.Size([150, 100])


#Models

In [0]:

class EmbedModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim, range_times, range_coors, range_words):
        super(EmbedModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linearTimes = nn.Linear(2 * embedding_dim, max(range_times)-min(range_times)+1, bias=False)
        self.linearCoors = nn.Linear(2 * embedding_dim, max(range_coors)-min(range_coors)+1, bias=False)
        self.linearWords = nn.Linear(2 * embedding_dim, max(range_words)-min(range_words)+1, bias=False)
        self.range_times = range_times
        self.range_coors = range_coors
        self.range_words = range_words
        
    def forward(self, inputs, target):
      
      
        embeds = self.embeddings(inputs).view((1, -1))
        
        
        if target in self.range_times:
           out = F.relu(self.linearTimes(embeds))


        elif target in self.range_coors:
            out = F.relu(self.linearCoors(embeds))
                      
                        

        else:#target in range_words
            out = F.relu(self.linearWords(embeds))
        
        log_probs = F.log_softmax(out, dim=1)

        return log_probs


class EmbedModelBatch(nn.Module):

    def __init__(self, vocab_size, embedding_dim, range_times, range_coors, range_words, batch_size):
        super(EmbedModelBatch, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linearTimes = nn.Linear(embedding_dim, max(range_times)-min(range_times)+1, bias=False)
        self.linearCoors = nn.Linear(embedding_dim, max(range_coors)-min(range_coors)+1, bias=False)
        self.linearWords = nn.Linear(embedding_dim, max(range_words)-min(range_words)+1, bias=False)
        self.range_times = range_times
        self.range_coors = range_coors
        self.range_words = range_words
        self.batch_size = batch_size
    
    def forward(self, inputs, target):
      
        #print(target)
        embeds = self.embeddings(inputs)
        print(embeds.shape)
        print(torch.sum(embeds[:], dim=0).shape)
        
        
        embeds = embeds.view((self.batch_size,-1))
        
        
        if target == 0:
           out = F.relu(self.linearTimes(embeds))


        elif target == 1:
            out = F.relu(self.linearCoors(embeds))
                      
                        

        else:#target in range_words
            out = F.relu(self.linearWords(embeds))
        
        log_probs = F.log_softmax(out, dim=1)

        return log_probs







class SimpleEmbedding():

    def __init__(self):
        pass


    def transform_as_nparray(self, tuples_train):

        output = []
        for (aux_date_idx, aux_coor_idx, text) in tuples_train:
            for word_numb in text:
                _tuple = (aux_date_idx, aux_coor_idx, word_numb)
                output.append(_tuple)
        return np.array(output)


    def get_input_layer(self, data, vocabulary_size):
        x = torch.zeros(vocabulary_size).float()
        for i in data:
            x[i] = 1.0
        return x

    def fit(self, tuples_train, embedding_dims, num_epochs = 11):

        xytrain = self.transform_as_nparray(tuples_train)
        vocabulary_size = int(max(list(xytrain[:, -1]))) + 1

        range_times = range(int(max(list(xytrain[:, -3]))) + 1)
        range_coor = range(int(max(list(xytrain[:, -3]))) + 1, int(max(list(xytrain[:, -2]))) + 1)
        range_words = range(int(max(list(xytrain[:, -2]))) + 1, vocabulary_size)

        W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
        W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
        learning_rate = 0.001
        numexamples = xytrain.shape[0]
        print('test')
        print(numexamples)
        numexamples = 1000
        
        
        
        
        
        for epo in range(num_epochs):
            loss_val = 0
            for tuple in xytrain[:numexamples]:
                # print(tuple)
                for i in range(3):
                    target = tuple[i]
                    data = [tuple[j] for j in range(3) if j != i]



                    #x = Variable(self.get_input_layer(data, vocabulary_size)).type(_type)

                    #z1 = torch.matmul(W1, x)
                    #z1 = W1[:, Variable(torch.from_numpy(np.array(data)).long()).type(_type)]
                    #z1 = W1[:, np.array(data)]
                    
                    if target in range_times:
                        y_true = Variable(torch.from_numpy(np.array([target])).long()).type(_typelong)
                        z2 = torch.matmul(W2[range_times], W1[:, np.array(data)])
                        # print(max_date_idx+1)

                    elif target in range_coor:
                        y_true = Variable(torch.from_numpy(np.array([target - min(range_coor)])).long()).type(_typelong)
                        z2 = torch.matmul(W2[range_coor], W1[:, np.array(data)])
                        # print(max_coor_idx+1)

                    else:#target in range_words
                        y_true = Variable(torch.from_numpy(np.array([target - min(range_words)])).long()).type(_typelong)

                        z2 = torch.matmul(W2[range_words], W1[:, np.array(data)])
                        # print(vocabulary_size-(max_coor_idx+1+max_date_idx+1))

                    # print(y_true)
                    # print(z2.shape)

                    log_softmax = F.log_softmax(z2, dim=0)

                    loss = F.nll_loss(log_softmax.view(1, -1), y_true)
                    loss_val += loss.data[0]
                    loss.backward()
                    #optimizer.step()


                    W1.data -= learning_rate * W1.grad.data
                    W2.data -= learning_rate * W2.grad.data

                    W1.grad.data.zero_()
                    W2.grad.data.zero_()

        #    if epo % 10 == 0:
            print(f'Loss {loss_val/numexamples}')
        self.W = torch.cat([W1.transpose(0, 1), W2], 1).detach().data.numpy()
    
    
    

    def next_batch(self):
              
        numexamples = self.xytrain.shape[0]
        
        positions = random.sample(range(numexamples),self.batch_size)
        
        target = random.sample(range(3),1)
        inputs = [j for j in range(3) if j != target[0]]
        
        
        x = self.xytrain[positions][:,inputs]
        y = self.xytrain[positions][:,target[0]]
        #print(y)
        y = y-self.min_ranges[target[0]]
        #print(-self.min_ranges[target[0]])
        #print(y)
        return x, y, target[0]
    
    def fit_embed_batch(self, tuples_train, embedding_dims, num_epochs = 11, batch_size = 100):
        self.batch_size = batch_size

        self.xytrain = self.transform_as_nparray(tuples_train)
        
        vocabulary_size = int(max(list(self.xytrain[:, -1]))) + 1

        range_times = range(int(max(list(self.xytrain[:, -3]))) + 1)
        range_coors = range(int(max(list(self.xytrain[:, -3]))) + 1, int(max(list(self.xytrain[:, -2]))) + 1)
        range_words = range(int(max(list(self.xytrain[:, -2]))) + 1, vocabulary_size)
        
        self.min_ranges = [min(range_times), min(range_coors), min(range_words)]
       
        numexamples = self.xytrain.shape[0]
        print(numexamples)
        #numexamples = 500000
        
        
        
        losses = []
        #loss_function = nn.NLLLoss()
        loss_function = torch.nn.CrossEntropyLoss()
        model = EmbedModelBatch(vocabulary_size, embedding_dims, range_times, range_coors, range_words, self.batch_size).to(device)
        optimizer = optim.SGD(model.parameters(), lr=0.001)

        for epoch in range(num_epochs):
            total_loss = 0
            
            
            for _ in range(3*numexamples//self.batch_size):
                x, y, target = self.next_batch()
                
                x = torch.tensor(x, dtype=torch.long)
                y = torch.tensor(y, dtype=torch.long)
                model.zero_grad()

                log_probs = model(x.to(device), target)

                                
                    
                y = torch.tensor(y, dtype=torch.long)
                #print(log_probs)
                #print(y)
                    
                #print(log_probs.shape)
                #print(y.shape)
                loss = loss_function(log_probs, y.to(device))

                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            losses.append(total_loss/numexamples)
            
            print(total_loss/numexamples)  # The loss decreased every iteration over the training data!
        self.W = model.embeddings.weight.data.cpu().numpy()
        print(self.W[0])
    
    
    def fit_embed(self, tuples_train, embedding_dims, num_epochs = 11):

        xytrain = self.transform_as_nparray(tuples_train)
        vocabulary_size = int(max(list(xytrain[:, -1]))) + 1

        range_times = range(int(max(list(xytrain[:, -3]))) + 1)
        range_coors = range(int(max(list(xytrain[:, -3]))) + 1, int(max(list(xytrain[:, -2]))) + 1)
        range_words = range(int(max(list(xytrain[:, -2]))) + 1, vocabulary_size)
        self.min_ranges = [min(range_times), min(range_coors), min(range_words)]
       
        numexamples = xytrain.shape[0]
        print(numexamples)
        #numexamples = 100
        
        
        
        losses = []
        loss_function = nn.NLLLoss()
        model = EmbedModel(vocabulary_size, embedding_dims, range_times, range_coors, range_words).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(num_epochs):
            total_loss = 0
            
            
            for tuple in xytrain[:numexamples]:
                # print(tuple)
                for i in range(3):
                    loss_steps = []
                    target = tuple[i]
                    # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
                    # into integer indices and wrap them in tensors)
                    inputs = [tuple[j] for j in range(3) if j != i]
                    
                    context_idxs = torch.tensor(inputs, dtype=torch.long)
                    

                    # Step 2. Recall that torch *accumulates* gradients. Before passing in a
                    # new instance, you need to zero out the gradients from the old
                    # instance
                    model.zero_grad()

                    # Step 3. Run the forward pass, getting log probabilities over next
                    # words
                    log_probs = model(context_idxs.to(device), target)

                    # Step 4. Compute your loss function. (Again, Torch wants the target
                    # word wrapped in a tensor)
                    
                    if target in range_times:
                        real_index_target = target
                        

                    elif target in range_coors:
                        real_index_target = target - min(range_coors)
                        

                    else:#target in range_words
                        real_index_target = target - min(range_words)

                    
                    
                    y_true = torch.tensor([real_index_target], dtype=torch.long)
                    
                    
                    
                    loss = loss_function(log_probs, y_true.to(device))

                    # Step 5. Do the backward pass and update the gradient
                    loss.backward()
                    optimizer.step()

                    # Get the Python number from a 1-element Tensor by calling tensor.item()
                    total_loss += loss.item()
            losses.append(total_loss/numexamples)
            if epoch % (num_epochs//10) == 0:
              print(total_loss/numexamples)  # The loss decreased every iteration over the training data!
        self.W = model.embeddings.weight.data.cpu().numpy()
    
    
    
    
    def predict(self, t_idx, l_idx, w_idx):
        vec_t = self.W[t_idx].reshape(1, -1)
        vec_l = self.W[l_idx].reshape(1, -1)

        vec_w = np.average([self.W[idx].reshape(1, -1) for idx in w_idx], axis=0)

        vectors = [vec_t, vec_l, vec_w]
        for vec1, vec2 in itertools.combinations(vectors, r=2):

            if (cosine_similarity(vec1, vec2).shape != (1, 1)):
                print('cosine_similarity shape error')

        score = sum([cosine_similarity(vec1, vec2)[0][0] for vec1, vec2 in itertools.combinations(vectors, r=2)])
        return round(score, 6)


In [20]:
%%time
translate = {'w':'Text','l':'Location','t':'Time'}
trainsets = [trainST, trainLA, trainNY]
testsets = [testST, testLA, testNY]
namesets = ['Santiago','LA','NY']
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[]}
for train,test,name in list(zip(trainsets, testsets, namesets))[:1]:
    QE = QuantitativeEvaluator()
    
    dims = [100]
    
    model = SimpleEmbedding()
    
    
    for dim in dims:
        model.fit_embed_batch(train, embedding_dims=dim, num_epochs=100, batch_size = 150)
        for predict_type in 'wlt':
            QE.get_ranks(test, model, predict_type = predict_type)
            mrr1 = QE.compute_mrr()[0]
            print(mrr1)
            results[translate[predict_type]].append(mrr1)
        results['Dataset'].append(name)
        results['Model'].append(str(type(model))+'_'+str())
        df = pd.DataFrame(results)
                
        df.to_csv('resultsemb.df')
        uploaded = drive.CreateFile()
        uploaded.SetContentFile('resultsemb.df')
        uploaded.Upload()

1327127
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torch.Size([150, 2, 100])
torc

KeyboardInterrupt: ignored

In [52]:

%time
for train,test,name in list(zip(trainsets, testsets, namesets))[:1]:
    QE = QuantitativeEvaluator()
    
    dims = [10]
    
    model = SimpleEmbedding()
    
    
    for dim in dims:
        model.fit_embed_batch(train, embedding_dims=dim, num_epochs=2,batch_size=1000)
        path = 'model_dim_'+str(dim)+'_'+name
        torch.save(model, path)
        uploaded = drive.CreateFile()
        uploaded.SetContentFile(path)
        uploaded.Upload()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.06 µs
1327127
tensor([[-0.2848, -0.3738,  0.0541,  0.4920,  0.3677,  0.7600, -1.2157, -0.8996,
         -0.8480, -0.1504]], grad_fn=<EmbeddingBackward>)
0.018999799914893416
0.018842834003921675
[-0.28268117 -0.37250555  0.05357122  0.49386552  0.3666874   0.75614697
 -1.212168   -0.89712894 -0.8461886  -0.14885308]


In [0]:
%time
for train,test,name in list(zip(trainsets, testsets, namesets))[:1]:
    QE = QuantitativeEvaluator()
    
    dims = [20]
    
    model = SimpleEmbedding()
    
    
    for dim in dims:
        model.fit_embed_batch(train, embedding_dims=dim, num_epochs=10)
        path = 'model_dim_'+str(dim)+'_'+name
        torch.save(model, path)
        uploaded = drive.CreateFile()
        uploaded.SetContentFile(path)
        uploaded.Upload()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.3 µs
1327127


In [21]:
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[]}
for train,test,name in list(zip(trainsets, testsets, namesets)):
    QE = QuantitativeEvaluator()
    
    dims = [20,60,100,160,200]
    
    model = SimpleEmbedding()
    
    
    for dim in dims:
        model.fit_embed(train, embedding_dims=dim, num_epochs=3)
    
    
        for predict_type in 'wlt':
            QE.get_ranks(test, model, predict_type = predict_type)
            mrr1 = QE.compute_mrr()[0]
            print(mrr1)
            results[translate[predict_type]].append(mrr1)
        results['Dataset'].append(name)
        results['Model'].append(str(type(model))+'_'+str())
        df = pd.DataFrame(results)
                
        df.to_csv('resultsemb.df')
        uploaded = drive.CreateFile()
        uploaded.SetContentFile('resultsemb.df')
        uploaded.Upload()

1327127
17.30460192312956
16.043830757434367
15.524767640408278
0.2691
0.2695


KeyboardInterrupt: ignored